In [ ]:
import pandas as pd
from data_source import DataSource
#from experiments import Experiments
from model_inference import ModelInference
from model_training import ModelTraining
from preprocessing import Preprocessing

In [ ]:
name_target = 'NU_NOTA_MT'
name_id = 'NU_INSCRICAO'

In [ ]:
## Analisa os dados de teste e de treino
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
print(f'Train shape: {df_train.shape}, Test shape: {df_test.shape}')

if name_target not in df_train.columns :
    print('ERROR: dataframe de treino não possui target')
    
elif df_test.columns.size >= (df_train.drop(columns=name_target).columns.size)\
        and all(elem in df_test for elem in df_train.drop(columns=name_target).columns) :
    print('ALERT: dataframes de teste e treino ok')

elif all(df_train[df_test.columns].columns == df_test.columns) :
    print('DONE: convertendo colunas dataframe de treino igual ao de teste + target')
    df_train = pd.concat([df_train[df_test.columns], df_train[name_target]], axis=1)
    print('DONE: gravando dataframe de treino formatado')
    df_train.to_csv('../data/train.csv', index=False)
else :
    print('ERROR: colunas dos Data frames de treino e testes são diferentes')

print(df_train.info())
del df_test, df_train

In [ ]:
## Análise de dados nulos e zerados
df_train = data.read_data()
print(f'dados Nulos: {df_train[(df_train["TP_PRESENCA_LC"] == 0) | (df_train["TP_PRESENCA_LC"] == 2) ][name_target].unique()}')
print(f'dados Zerados: {df_train[(df_train["NU_NOTA_LC"] == 0)][name_target].unique()}')
del df_train

In [ ]:
lin_null = [('TP_PRESENCA_LC', 0), ('TP_PRESENCA_LC', 2)]
lin_zero = [('NU_NOTA_LC', 0)]
data = DataSource(name_id=name_id, name_target=name_target, rows_remove=(lin_null + lin_zero))

In [ ]:
pre = Preprocessing(data)

In [ ]:
#df_new = pre.process()
#df_new[0].info()

In [ ]:
#Experiments(data).run_experiment()

In [ ]:
model = ModelTraining(pre)

In [ ]:
trained_model = model.training()

In [ ]:
y_pred = ModelInference(trained_model).predict()

In [ ]:
df_answer = data.get_columns(name_columns=[name_id], is_train_stage=False)
df_answer[name_target] = y_pred.round(1)

df_removed = data.get_removed_rows(name_columns=[name_id], is_train_stage=False)
df_removed[name_target] = 0

df_answer = pd.concat([df_answer,df_removed])

In [ ]:
df_answer.to_csv('../answer.csv', index=False)